In [ ]:
import os
import json
from dotenv import load_dotenv
from langchain_core.documents import Document
from benchmark.protocol_parsing.risk import extract_and_save_procedure_risk_pairs

# Load environment variables
load_dotenv()

# Example protocol content with procedure-risk pairs
MOCK_PROTOCOL_PAGES = [
    Document(
        page_content="""
        5.1 Study Procedures
        
        Blood Collection: Blood samples will be collected at screening, baseline, and weeks 4, 8, and 12.
        Approximately 10-15 mL of blood will be drawn at each visit. Common risks associated with blood
        collection include pain, bruising, or redness at the needle site. Rarely, some participants may
        experience dizziness or fainting during blood collection.
        
        Physical Examination: A complete physical examination will be performed at screening and
        a targeted physical examination at subsequent visits. No significant risks are associated with
        physical examinations, though some participants may experience mild discomfort.
        """,
        metadata={"page_number": 23}
    ),
    Document(
        page_content="""
        5.2 Imaging Procedures
        
        MRI Scan: Participants will undergo MRI scans at baseline and at week 12. MRI scans will last
        approximately 45-60 minutes. Risks associated with MRI include loud noise that may cause
        temporary discomfort, claustrophobia due to the enclosed space, and potential discomfort from
        lying still for an extended period. Participants with metal implants cannot undergo MRI.
        
        X-ray: A chest X-ray will be performed at screening to rule out active tuberculosis.
        Risks include exposure to a small amount of radiation. The benefits of diagnostic information
        outweigh the minimal risks from radiation exposure.
        """,
        metadata={"page_number": 24}
    ),
    Document(
        page_content="""
        5.3 Study Drug Administration
        
        Study Drug Infusion: The study drug will be administered via intravenous infusion over 1 hour
        at Weeks 0, 4, and 8. Possible risks associated with the study drug include infusion reactions
        (fever, chills, rash, shortness of breath), headache, nausea, and fatigue. Serious but rare
        adverse events include severe allergic reactions and liver enzyme elevations.
        
        Placebo Administration: Participants randomized to the placebo arm will receive an identical-looking
        solution without the active ingredient. Risks related to placebo administration are limited to
        those associated with the IV infusion procedure itself, such as discomfort at the IV site.
        """,
        metadata={"page_number": 25}
    ),
    Document(
        page_content="""
        8.2 Risk Management
        
        Liver Function Tests: LFTs will be monitored throughout the study. If significant elevations
        occur, the study drug may be withheld according to protocol-specified criteria. Risks of LFT
        monitoring are limited to those associated with blood collection.
        
        Questionnaires: Participants will complete several questionnaires at each visit. Some questions
        may cause psychological discomfort. Participants will be informed that they can skip questions
        that make them uncomfortable.
        """,
        metadata={"page_number": 45}
    ),
    Document(
        page_content="""
        10.1 Safety Assessments
        
        Electrocardiogram (ECG): A 12-lead ECG will be performed at screening, baseline, and at weeks 4, 8, and 12.
        ECGs are non-invasive and pose no significant risks. Some participants may experience mild
        discomfort from electrode placement.
        
        Vital Signs: Blood pressure, pulse, respiratory rate, and temperature will be measured at each visit.
        No risks are associated with vital sign measurements.
        """,
        metadata={"page_number": 62}
    )
]

def main():
    print("Testing procedure-risk extraction pipeline...\n")
    
    # Define output path
    output_dir = "./extraction_results"
    os.makedirs(output_dir, exist_ok=True)
    output_path = f"{output_dir}/mock_procedure_risk_pairs.json"
    
    # Run the extraction pipeline
    results = extract_and_save_procedure_risk_pairs(
        documents=MOCK_PROTOCOL_PAGES,
        output_path=output_path,
        llm="gpt-4o-mini"  # Use a smaller model for testing
    )
    
    # Print some of the extracted pairs for verification
    print("\nExtracted Procedure-Risk Pairs Sample:")
    procedure_count = len(results["procedure_risk_pairs"])
    
    # Show detailed view of first 3 procedures
    procedures = list(results["procedure_risk_pairs"].keys())
    for i, procedure in enumerate(procedures[:3], 1):
        risks = results["procedure_risk_pairs"][procedure]
        source_pages = results["sources"][procedure]
        
        print(f"\n{i}. {procedure} (from page(s) {source_pages}):")
        for j, risk in enumerate(risks, 1):
            print(f"   {j}. {risk}")
    
    # If more than 3 procedures were found, mention how many more
    if procedure_count > 3:
        print(f"\n...and {procedure_count - 3} more procedures")
    
    print(f"\nComplete results saved to: {output_path}")


In [ ]:

main() 